# Week 8 - Generators and `for`
_GitHub ipynb viewer does not render html in markdown cells_

- **generator**

> A function which returns a generator iterator. It looks like a normal function except that it contains `yield` expressions for producing a series of values usable in a for-loop or that can be retrieved one at a time with the `next()` function.
Usually refers to a generator function, but may refer to a generator iterator in some contexts. In cases where the intended meaning isn’t clear, using the full terms avoids ambiguity.

- **generator iterator**

> An object created by a generator function. \
Each yield temporarily suspends processing, remembering the location execution state (including local variables and pending try-statements). When the generator iterator resumes, it picks up where it left off (in contrast to functions which start fresh on every invocation).

## [**Yield expressions and generator methods**](https://docs.python.org/3/reference/expressions.html#yield-expressions)

Simple fibonacci generator

In [127]:
def fibgen():
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b
        
        
# 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55
generator = fibgen()

print(next(generator))
# 0 is yielded value

print(generator.__next__())

for number in fibgen():
    if number >= 50:
        break
    print(number)

0
1
0
1
1
2
3
5
8
13
21
34


**``__next__()`` and ``send(value)`` interaction**

In [132]:
def simplegen():
    n = 0
    while True:
        n += 1
        cached_n = n
        n = yield n  # __next__() yields n and sets the yield to None,
                     # so n becomes None as a result
        if n is None:  # we restore n after the __next__() call
            n = cached_n
        

gen = simplegen()

for i in range(3):
    for _ in range(2):
        print(f'__next__(): \t\t\t\t {next(gen)}')
    value_to_send = i*10
    print(f'send() argument \t\t\t {value_to_send}')
    print(f'send() returns the next value yielded:\t {gen.send(value_to_send)}')
    
gen.close()
try:
    next(gen)
except StopIteration:
    print('No')
    pass

__next__(): 				 1
__next__(): 				 2
send() argument 			 0
send() returns the next value yielded:	 1
__next__(): 				 2
__next__(): 				 3
send() argument 			 10
send() returns the next value yielded:	 11
__next__(): 				 12
__next__(): 				 13
send() argument 			 20
send() returns the next value yielded:	 21
No


In [ ]:
def A():
    print('A')
def B():
    print('B')
def C():
    print('C')
    
list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

# Outline
_Вам следует отработать описанные ниже пункты и убедиться, что Вы знаете каждый из них._
## Основное
1. Концепция генератора. 
Чем он отличается от того же `list`, какие дает преимущества.

2. Что происходит при вызове `next(iterator)`?

3. Какое поведение ожидается от генератора, когда он по логике программы больше ничего не должен возвращать (т.е. каким образом генератор сигнализирует об окончании последовательности)?

4. `map`, `zip`, `filter`, `enumerate` - знать эти функции и понимать, что они делают.

5. Библиотека `itertools` - знать, что она есть.
Представлять, для решения каких задач ее можно использовать.
Суметь найти документацию по ней и сориентироваться в документации.

6. Понимать, как написать собственный генератор.
Знать, как работает в коде слово `yield`, чем оно отличается от `return`.
Суметь написать свой генератор.



```







```

---

# Домашнее задание по неделе 8
Число в скобках - количество баллов.
_В этой неделе все ДЗ взято с [соответствующей лабы](http://cs.mipt.ru/advanced_python/lessons/lab08.html)_


Везде, где предлагается протестировать программу, используйте `pytest`. 
Также ожидается, что из успешного прохождения тестов **действительно** можно сделать вывод о работоспособности программы - старайтесь покрыть краевые случаи, а также тестируйте на разных данных.

## Генераторы и цикл for

<center>

### 1
</center>

> (2)
Вам дана функция на языке python:


>```python
def print_map(function, iterable):
    for i in iterable:
        print(function(i))
>```

> Требуется переписать данную функцию не используя цикл for.

In [68]:
import numpy as np


arr = np.array([0, np.pi, 3*np.pi/2])

def print_map(function, iterable):
    for i in iterable:
        print(function(i))
        
        
print_map(np.sin, arr)

0.0
1.2246467991473532e-16
-1.0


In [69]:
def print_map(function, iterable):
    print(*(map(function, iterable)), sep='\n')
    
    
print_map(np.sin, arr)

0.0
1.2246467991473532e-16
-1.0


<center>
    
### 2
</center>
    
> (3) Напишите генератор, выводящий первые n чисел Фибоначчи.
(1) Напишите несколько тестов на генератор.

In [43]:
%%writefile tests/test_task_2.py

def fibgen():
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b


def fib(n):
    f = fibgen()
    a = []
    for _ in range(n):
        a.append(next(f))
    return a


def test_fib():
    assert fib(10)[0] == 0
    assert fib(11)[3] == 2
    assert fib(4) == [0, 1, 1, 2]

Overwriting tests/test_task_2.py


<center>

### 3
<center>
    
> (3) Реализуйте аналог функций zip, map, enumerate.
(1) Напишите несколько тестов на нескольких данных.

[**Built-in Functions**](https://docs.python.org/3/library/functions.html#built-in-functions)

In [93]:
%%writefile tests/test_task_3.py

import numpy as np


def my_zip(*iterables):
    sentinel = object()
    iterators = [iter(it) for it in iterables]
    while iterators:
        result = []
        for it in iterators:
            elem = next(it, sentinel)
            if elem is sentinel:
                return
            result.append(elem)
        yield tuple(result)

        
def test_zip():
    a = ['a', 'b', 'c']
    b = [1, 2, 3]
    for pair in zip(my_zip(a, b), zip(a, b)):
        assert pair[0] == pair[1]


def my_map(func, iterable):
    for i in iterable:
        yield func(i)
        
    
def test_map():
    f = np.sin
    it = [1, 2, 3]
    for pair in zip(my_map(f, it), map(f, it)):
        assert pair[0] == pair[1]


def my_enumerate(sequence, start=0):
    n = start
    for elem in sequence:
        yield n, elem
        n += 1
        
        
def test_enumerate():
    a = ['a', 'b', 'c']
    for pair in zip(my_enumerate(a), enumerate(a)):
        assert pair[0] == pair[1]

Overwriting tests/test_task_3.py


<center>
    
### 4
</center>
    
> 4. (3) Написать функцию, принимающую 2 списка и возвращающую декартово произведение (использовать `itertools.product`)

```python
def get_cartesian_product(a, b):
    raise RuntimeError("Not implemented")

assert get_cartesian_product([1, 2], [3, 4]) == [(1, 3), (1, 4), (2, 3), (2, 4)]
```

> (1) Напишите несколько тестов на нескольких данных.

In [62]:
%%writefile tests/test_task_4.py

from itertools import product


def get_cartesian_product(a, b):
    return list(product(a, b))


def test_product():
    assert get_cartesian_product([1, 2], [3, 4]) == [(1, 3), (1, 4), (2, 3), (2, 4)]
    assert get_cartesian_product([1, 2], [3]) == [(1, 3), (2, 3)]
    assert get_cartesian_product([1, 2, 3, 4], [5, 6, 7, 8]) == [(1, 5), (1, 6), (1, 7), (1, 8),
                                                                 (2, 5), (2, 6), (2, 7), (2, 8),
                                                                 (3, 5), (3, 6), (3, 7), (3, 8),
                                                                 (4, 5), (4, 6), (4, 7), (4, 8)]

Overwriting tests/test_task_4.py


<center>

### 5
</center>

> (3) Написать функцию, принимающую строку `s` и число `n` и возвращающую всевозможные перестановки из `n` символов в `s` строке в **лексикографическом** порядке (использовать `itertools.permutations`)

```python
def get_permutations(s, n):
    raise RuntimeError("Not implemented")

assert get_permutations("cat", 2) == ["ac", "at", "ca", "ct", "ta", "tc"]
```

> (1) Напишите несколько тестов на нескольких данных.

In [47]:
%%writefile tests/test_task_5.py

from itertools import permutations


def get_permutations(s, n):
    return sorted([''.join(x) for x in permutations(s, n)])


def test_permutations():
    assert get_permutations('cat', 2) == ['ac', 'at', 'ca', 'ct', 'ta', 'tc']
    assert get_permutations('dog', 1) == ['d', 'g', 'o']

Overwriting tests/test_task_5.py


<center>
    
### 6
</center>

> (3) Реализовать функцию `get_combinations`. Должна принимать строку `s` и число `k` и возвращать все возможные комбинации из символов в строке `s` с длинами <= `k` (использовать `itertools.combinations`)

```python
def get_combinations(s, n):
    raise RuntimeError("Not implemented")

assert get_combinations("cat", 2) == ["a", "c", "t", "ac", "at", "ct"]
```

> (1) Напишите несколько тестов на нескольких данных.

In [49]:
%%writefile tests/test_task_6.py

from itertools import combinations


def get_combinations(s, n):
    a = sorted([''.join(sorted(x)) for i in range(1, n+1) for x in combinations(s, i)])
    a.sort(key=lambda x: len(x))
    return a


def test_combinations():
    assert get_combinations('cat', 2) == ['a', 'c', 't', 'ac', 'at', 'ct']
    assert get_combinations('dog', 3) == ['d', 'g', 'o', 'dg', 'do', 'go', 'dgo']

Overwriting tests/test_task_6.py


<center>
    
### 7
</center>

> (3) Функция должна принимать строку `s` и число `k` и возвращать все возможные комбинации из символов в строке `s` с длинами, равными `k`, с повторениями (использовать `itertools.combinations_with_replacement`)

```python
def get_combinations_with_r(s, n):
    raise RuntimeError("Not implemented")

assert get_combinations_with_r("cat", 2) == ["aa", "ac", "at", "cc", "ct", "tt"]
```

> (1) Напишите несколько тестов на нескольких данных.

In [71]:
%%writefile tests/test_task_7.py

from itertools import combinations_with_replacement


def get_combinations_with_r(s, n):
    return sorted([''.join(sorted(x)) for x in combinations_with_replacement(s, n)])

    
def test_get_combinations_with_r():
    assert get_combinations_with_r('cat', 2) == ['aa', 'ac', 'at', 'cc', 'ct', 'tt']
    assert get_combinations_with_r('dog', 3) == ['ddd', 'ddg', 'ddo', 'dgg', 'dgo', 
                                                 'doo', 'ggg', 'ggo', 'goo', 'ooo']
    assert get_combinations_with_r('dog', 8) == ['dddddddd', 'dddddddg', 'dddddddo', 'ddddddgg',
                                                 'ddddddgo', 'ddddddoo', 'dddddggg', 'dddddggo',
                                                 'dddddgoo', 'dddddooo', 'ddddgggg', 'ddddgggo',
                                                 'ddddggoo', 'ddddgooo', 'ddddoooo', 'dddggggg',
                                                 'dddggggo', 'dddgggoo', 'dddggooo', 'dddgoooo',
                                                 'dddooooo', 'ddgggggg', 'ddgggggo', 'ddggggoo',
                                                 'ddgggooo', 'ddggoooo', 'ddgooooo', 'ddoooooo',
                                                 'dggggggg', 'dggggggo', 'dgggggoo', 'dggggooo',
                                                 'dgggoooo', 'dggooooo', 'dgoooooo', 'dooooooo',
                                                 'gggggggg', 'gggggggo', 'ggggggoo', 'gggggooo',
                                                 'ggggoooo', 'gggooooo', 'ggoooooo', 'gooooooo',
                                                 'oooooooo']

Overwriting tests/test_task_7.py


<center>
    
### 8
</center>

> (3) Написать функцию, которая подсчитывает количество подряд идующих символов в строке (использовать `itertools.groupby`)

```python
def compress_string(s):
    raise RuntimeError("Not implemented")

assert compress_string('1222311') == [(1, 1), (3, 2), (1, 3), (2, 1)]
```

> (1) Напишите несколько тестов на нескольких данных.

In [76]:
%%writefile tests/test_task_8.py

from itertools import groupby


def get_combinations_with_r(s, n):
    return sorted([''.join(sorted(x)) for x in combinations_with_replacement(s, n)])

    
def compress_string(s):
    groups = []
    uniquekeys = []
    for k, g in groupby(s):
        groups.append(len(list(g)))
        uniquekeys.append(int(k))
    return list(zip(groups, uniquekeys))

        
def test_compress_string():
    assert compress_string('1222311') == [(1, 1), (3, 2), (1, 3), (2, 1)]
    assert compress_string('12223111') == [(1, 1), (3, 2), (1, 3), (3, 1)]
    compress_string('144423455553333532341') == [(1, 1), (3, 4), (1, 2), (1, 3),
                                                 (1, 4), (4, 5), (4, 3), (1, 5),
                                                 (1, 3), (1, 2), (1, 3), (1, 4),
                                                 (1, 1)]

Overwriting tests/test_task_8.py


<center>
    
### 9
</center>

> (4) В функцию передается список списков. Нужно вернуть максимум, который достигает выражение `a_1^2 + a_2^2 + ... + a_n^2) % m`. Где `a_i` - максимальный элемент из i-ого списка (использовать функцию из `itertools`)

```python
def maximize(lists, m):
    raise RuntimeError("Not implemented")

lists = [
    [5, 4],
    [7, 8, 9],
    [5, 7, 8, 9, 10]
]
assert maximize(lists, m=1000) == 206
```

> В примере = 206, так как это максимум от суммы `(a_21 + a_22 + a_23) % 1000`.

> (2) Напишите несколько тестов на нескольких данных.

In [133]:
%%writefile tests/test_task_9.py

from itertools import accumulate


def maximize(lists, m):
    squared_maxes = [max(lst)**2 for lst in lists]
    polynomials = [p for p in accumulate(squared_maxes)]
    
    return max([x % m for x in polynomials])


def test_maximize():
    lists = [
        [5, 4],
        [7, 8, 9],
        [5, 7, 8, 9, 10]
    ]
    assert maximize(lists, m=1000) == 206
    
    lists = [
            [1000, 4],
            [7, 8, 9],
            [5, 7, 8, 9, 10]
        ]
    assert maximize(lists, 1000) == (1000 ** 2 + 9 ** 2 + 10 ** 2) % 1000
    
    lists = [
            [1, 1],
            [1, 2],
            [0, 3]
        ]
    assert maximize(lists, 10) == (1 ** 2 + 2 ** 2) % 10

Overwriting tests/test_task_9.py


In [134]:
!pytest -vv -p no:warnings

============================= test session starts =============================
platform win32 -- Python 3.8.6, pytest-6.1.1, py-1.9.0, pluggy-0.13.1 -- C:\DEV\anaconda\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\barklan\DataScience\mipt_adv_py\w8
collecting ... collected 11 items

tests/test_task_2.py::test_fib PASSED                                    [  9%]
tests/test_task_3.py::test_zip PASSED                                    [ 18%]
tests/test_task_3.py::test_map PASSED                                    [ 27%]
tests/test_task_3.py::test_enumerate PASSED                              [ 36%]
tests/test_task_4.py::test_product PASSED                                [ 45%]
tests/test_task_5.py::test_permutations PASSED                           [ 54%]
tests/test_task_6.py::test_combinations PASSED                           [ 63%]
tests/test_task_7.py::test_get_combinations_with_r PASSED                [ 72%]
tests/test_task_8.py::test_compress_string PASSED                   